In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('./mushrooms.csv')
data.head()
# data.info()
# data.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
# X = data.drop(['class'], axis=1)
# Y = data['class']
# # onehot encoding for x
# enc = OneHotEncoder().fit(X)
# labelEncoding for y

#split into training and testing data

In [4]:
# take a look at all of the unique values for each column
for col in data.columns:
    print(col, ": ", len(data[col].unique()), " | ", data[col].unique())

# note: there are 2480 values of '?' in the stalk-root column
print(data['stalk-root'].value_counts())

# replace those '?' with NaN
data = data.replace('?', np.nan)

# check to make sure they were properly replaced
print(data['stalk-root'].value_counts())

class :  2  |  ['p' 'e']
cap-shape :  6  |  ['x' 'b' 's' 'f' 'k' 'c']
cap-surface :  4  |  ['s' 'y' 'f' 'g']
cap-color :  10  |  ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
bruises :  2  |  ['t' 'f']
odor :  9  |  ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
gill-attachment :  2  |  ['f' 'a']
gill-spacing :  2  |  ['c' 'w']
gill-size :  2  |  ['n' 'b']
gill-color :  12  |  ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
stalk-shape :  2  |  ['e' 't']
stalk-root :  5  |  ['e' 'c' 'b' 'r' '?']
stalk-surface-above-ring :  4  |  ['s' 'f' 'k' 'y']
stalk-surface-below-ring :  4  |  ['s' 'f' 'y' 'k']
stalk-color-above-ring :  9  |  ['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']
stalk-color-below-ring :  9  |  ['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']
veil-type :  1  |  ['p']
veil-color :  4  |  ['w' 'n' 'o' 'y']
ring-number :  3  |  ['o' 't' 'n']
ring-type :  5  |  ['p' 'e' 'l' 'f' 'n']
spore-print-color :  9  |  ['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']
population :  6  |  ['s' 'n' 'a' 'v' 'y' 'c']
habitat :  7  | 

In [5]:
data['class'].value_counts()

e    4208
p    3916
Name: class, dtype: int64

In [6]:
print("shape: ", data.shape)

# encoding the data
labelencoder=LabelEncoder()
for col in data.columns:
    data[col] = labelencoder.fit_transform(data[col])
    
data

shape:  (8124, 23)


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [7]:
training_data, testing_data = train_test_split(data, test_size=0.2, random_state=25)


# seperate the dataset into features and target
X = training_data.drop(['class'],axis=1)
y = training_data['class']
print(y.head())
print(X.shape)
X.corr

# use a tree-based feature selection to remove irrelevant features
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X.values, y.values)
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X.values)
X_new.shape           
new = model.transform(X.values)
X_new.shape           
X_new

6635    1
4       0
1217    0
4359    1
646     0
Name: class, dtype: int64
(6499, 22)


array([[0, 7, 0, ..., 1, 0, 7],
       [0, 5, 1, ..., 2, 0, 3],
       [0, 5, 1, ..., 0, 0, 2],
       ...,
       [0, 2, 0, ..., 1, 0, 7],
       [1, 3, 1, ..., 2, 4, 3],
       [1, 5, 0, ..., 2, 0, 7]])

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
# import matplotlib

model_tree = DecisionTreeClassifier()

X_train = X_new
y_train = y


model_tree = model_tree.fit(X_train, y_train)

tree.plot_tree(model_tree)

ModuleNotFoundError: No module named 'matplotlib'